In [2]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [4]:
# parameters
customer_id = '5'
formatted_attribute = 'diameter'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Multiple Buckets"""

# regex_pattern = r'(?i)(\d+\.\d*\s?\\\"\s?i?n?\s?diam)|(\d*\s?\\\"\s?i?n?\s?diam)|()'
regex_pattern_NA = r'(?i)(\d+\s?\\\"\s?diam)|(\d*\\?\"\s?D)|(\d*\.?\d*\\?\"\s?D")|()'
regex_pattern = r"(?i)(\d+\s?\\\"\s?diam)|(\d+\.\d*\s?\\\"\s?diam)|()"
regex_pattern_inch = r"(?i)(\d+.?Inch)|()"
regex_pattern_wreath = r"(?i)(Wreath)|()"

value='%n/a%'
customer_name='%bedbathandbeyond%'
strategy_version_input=677
attribute_id_input=2730

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [5]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]

In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df))
df = df[df['bucket_id'].isin(buckets)]
len(df)

3985


2654

# General Diameter and Round

In [31]:
regex_pattern=r'(?i)(\d+\-\d*\/\d*.?\\?\".?diam)|(\d*\\?\".?diam)|(\d*\.\d*.?\".?diam)|(\d*\\\"\s?round)|(\d*\.\d*.?\'.?diam)|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df['matches'][0:5]

0               []
1               []
2               []
3               []
4    [2.83\" diam]
Name: matches, dtype: object

In [37]:
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

In [38]:
df['matches']= df['matches'].apply(lambda x: sorted(x, key=natural_key))
df['matches'][0:50]

0                                    []
1                                    []
2                                    []
3                                    []
4                         [2.83\" diam]
5                                    []
6                                    []
7                                    []
8                           [20\" diam]
9                                    []
10                                   []
11                                   []
12                                   []
13                                   []
14                                   []
15                          [16\" diam]
17                                   []
18                 [7\" diam, 9\" diam]
19    [43\" diam, 48\" diam, 55\" diam]
20                                   []
21                                   []
22                                   []
23                          [61\" diam]
24                                   []
25                                   []


In [30]:
regex_pattern=r'(?i)(\d+\-\d*\/\d*.?\\?\".?L.?)|(\d*\\?\".?len)|(\d*\.\d*.?\".?d)|(\d*\\\"\s?round)|(\d*\.\d*.?\'.?D)|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df['matches']= df['matches'].apply(lambda x: sorted(x, key=natural_key))
df['matches'].explode().value_counts()

3.5\" d        26
7.8\" d        18
8.5\" d        14
10.5\" d       14
2.93\" d       10
1.5\" d        10
3.22\" d        9
2.5\" d         9
2.8\" d         9
4.5\" d         8
3.96\" d        8
2.75\" d        7
2.5\" D         7
1.5\" D         7
5.5\" d         7
7.25\" d        6
4.75\" d        6
1.75\" d        6
5.25\" d        6
3.9\" d         6
7.5\" d         5
3.14\" d        5
7.5' d          5
3.25\" d        5
0.32\" D        5
4.5\" D         5
18.5\" d        5
0.5\" D         4
9\" round       4
0.31\" D        4
5.75\" d        4
3.35\" d        4
7\" round       4
15.35\" D       4
21.5\" D        3
20.5\" d        3
3.75\" d        3
2.75\" D        3
11.5\" D        3
4.25\" d        3
12.99\" d       3
8.75\" d        3
9.5\" d         3
3.15\" d        3
3.4\" d         2
6.5\" d         2
0.75\" D        2
1.25\" D        2
3.7\" D         2
6.3\" d         2
6.88\" d        2
4.55\" d        2
3.63\" d        2
22.63\" d       2
.25\" D         2
12\" round

In [152]:
matches=df[df['matches'].astype(str)!='[]']
matches[curation_col]=df['matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\" diam',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" round',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'Round','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'12"','12 in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'30 in\"\,\"30 in','30 in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [153]:
cooling=df[df['buckets'].astype(str)=='Cooling & Ventilation Equipment']
regex_pattern_cool=r'(?i)(\d+\-?\d*\/?\d*\\\"\s?W)|(\d*\.\d*\\\"\s?W)|(\d+\-?\s?Inch)|()'
cooling['matches'] = cooling['long_desc'].apply(lambda x: re_extract(regex_pattern_cool, x))
cooling['name_matches'] = cooling['product_name'].apply(lambda x: re_extract(regex_pattern_cool, x))
cooling.head(1)

product_id external_id  \
1     3807829    47513358   

                                                               product_name  \
1  Concord Heritage Series 52-Inch 3-Light Ceiling Fan in Oil Rubbed Bronze   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
1  The Heritage Series Ceiling Fan offers simple elegance and effective design ideal for any room in your home. 5-blade design with 3 speed forward and reverse for optimal air circulation. 3 light shades offer ample lighting to brighten up any room. The Heritage Series 52-Inch Ceiling Fan by Concord is an elegant addition to any room, offering effective air circulation and downward lighting to brighten your space Circulates air in spaces up to 225 sq. ft., providing a breeze effect Provides ample downward lighting for bedrooms, living spaces, dining rooms and more 153 mm x 13 mm motor 3 speed forward and 3 speed reverse provides superior air circulation for all living spaces 5 blades and 3 bell shape glass shades Pull chain on/off and remote operation Fans can be installed in ceilings with up to a 45 degree slope Comes with a dual mount for flush mount application RM-101 universal remote adaptable (not included) Indoor use only Requires three 60-watt E12 bulbs (included) Steel construction Wipe clean Measures 52\" L x 52\" W x 16.88\" H Weighs 18 lb. UL listed All Concord motors are covered by a limited lifetime manufacturer's warranty Imported Model 52HEH5EBB (Polished Brass); 52HEH5EWH (White); 52HEH5EORB (Oil Rubbed Bronze); 52HEH5ESN (Satin Nickel)   

   customer_id attributes                          buckets  bucket_id values  \
1            5       None  Cooling & Ventilation Equipment       7357   None   

             matches name_matches  
1  [52-Inch, 52\" W]    [52-Inch]

In [102]:
matches_cool=cooling[cooling['name_matches'].astype(str)!='[]']
matches_cool[curation_col]=cooling['name_matches'].astype(str)
matches_cool[curation_col]=matches_cool[curation_col].apply(lambda x: f'"{x}"')
matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r'\-','',str(measure)))
matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r'\" W',' in',str(measure)))
matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r'Inch',' in',str(measure)))
# matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r'12"','12 in',str(measure)))
# matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
# matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))
matches_cool[curation_col]=matches_cool[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [147]:
na_upload=df[(df['matches'].astype(str)=='[]')&(df['buckets'].astype(str)!='Cooling & Ventilation Equipment')][0:500]
na_upload[curation_col]=r'n/a'

# Trees

In [40]:
regex_pattern=r'(?i)(\d*\\?\".?diam)|(\d*\.\d*.?\".?diam)'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
tree=df[df['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
tree['matches']= tree['matches'].apply(lambda x: sorted(x, key=natural_key))
tree.head(1)

product_id external_id  \
7  1040311097    62928663   

                                                  product_name  \
7  Alpine 3-Piece Pre-Lit Christmas Tree Set with Clear Lights   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           long_desc  \
7  Decorate your home or office for the holiday season with this Alpine Pre-Lit 3-Piece Christmas Tree Set. The set includes 3 matching pre-lit Alpine trees with log-style bases in heights of 3', 4', and 5' to bring a festive atmosphere to your space. Log-style base stands 2\" wide tips Pre-lit with UL listed clear mini lights Set of 3 trees includes: 3' tree: 130 tips 50 clear mini lights No assembly required Measures 16\" diameter x 3\" H 4' tree: 250 tips 100 clear mini lights 3-piece assembly Measures 22\" diameter x 4' H 5' tree: 376 tips 150 clear mini lights 3-piece assembly Measures 27\" diameter x 5\" H PVC/metal/wood/glass bulbs/wire Imported   

   customer_id attributes                                   buckets  \
7            5       None  Trees, Shrubs, Topiaries, & Other Plants   

   bucket_id values                            matches  
7       8072   None  [16\" diam, 22\" diam, 27\" diam]

In [41]:
#import regex as re
matches=tree
matches[curation_col]=tree['matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\" diam',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\"xa0diam',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches
matches=matches[matches['matches'].astype(str)!='[]']
matches.head(1)

product_id external_id  \
7  1040311097    62928663   

                                                  product_name  \
7  Alpine 3-Piece Pre-Lit Christmas Tree Set with Clear Lights   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           long_desc  \
7  Decorate your home or office for the holiday season with this Alpine Pre-Lit 3-Piece Christmas Tree Set. The set includes 3 matching pre-lit Alpine trees with log-style bases in heights of 3', 4', and 5' to bring a festive atmosphere to your space. Log-style base stands 2\" wide tips Pre-lit with UL listed clear mini lights Set of 3 trees includes: 3' tree: 130 tips 50 clear mini lights No assembly required Measures 16\" diameter x 3\" H 4' tree: 250 tips 100 clear mini lights 3-piece assembly Measures 22\" diameter x 4' H 5' tree: 376 tips 150 clear mini lights 3-piece assembly Measures 27\" diameter x 5\" H PVC/metal/wood/glass bulbs/wire Imported   

   customer_id attributes                                   buckets  \
7            5       None  Trees, Shrubs, Topiaries, & Other Plants   

   bucket_id values                            matches  \
7       8072   None  [16\" diam, 22\" diam, 27\" diam]   

                  Q:diameter  
7  ["16 in","22 in","27 in"]

In [45]:
len(matches)


745

# Everything but trees

In [29]:
not_tree=df[df['buckets'].astype(str)!='Trees, Shrubs, Topiaries, & Other Plants']
not_tree=not_tree[not_tree['matches'].astype(str)!='[]']
not_tree['matches']= not_tree['matches'].apply(
    lambda x: sorted(x, key=natural_key))
not_tree.head(1)

product_id external_id                                product_name  \
6   817187044    62289269  Waterford® Lismore Diamond Red 8-Inch Vase   

                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
6  Revel in the extravagant beauty of Waterford's stunning Lismore Diamond Red Vase. Chic and modern, it features Lismore's intricate diamond and wedge cuts in brilliant crystal paired with a rich, ruby red that adds colorful style to any décor. Luxuriously rich color paired with brilliant Waterford crystal Modern update of the beloved Lismore pattern Lead crystal Hand wash Measures 6\" diameter x 8\" H Imported   

   customer_id attributes buckets  bucket_id values     matches  
6            5       None   Vases       7935   None  [6\" diam]

In [32]:
matches_not_tree=not_tree
matches_not_tree[curation_col]=not_tree['matches'].astype(str)
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda x: f'"{x}"')
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda measure: re.sub(r'\\" diam',' in',str(measure)))
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda measure: re.sub(r'\.125','.13',str(measure)))
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))
matches_not_tree[curation_col]=matches_not_tree[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [34]:
matches_not_tree=matches_not_tree[matches_not_tree[curation_col].astype(str)!='["1.5 in"]']

matches_not_tree[curation_col].explode().value_counts()

["14 in"]                         26
["15 in"]                         24
["16 in"]                         22
["9 in"]                          18
["12 in"]                         15
["13 in"]                         15
["8 in"]                          15
["10 in"]                         15
["5.5 in"]                        11
["3.5 in"]                        11
["10.5 in"]                        9
["8.5 in"]                         9
["24 in"]                          8
["7 in"]                           6
["6 in"]                           6
["3 in"]                           5
["4.73 in","5.52 in"]              5
["17 in"]                          5
["11 in"]                          4
["19 in"]                          3
["12.75 in"]                       3
["36 in"]                          3
["48 in"]                          3
["4.75 in"]                        3
["20 in"]                          3
["4 in"]                           3
["4.5 in"]                         3
[

In [46]:
len(matches_not_tree)

348

# Wreaths

In [ ]:
df['wreath_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_wreath, x))
x=df[df['matches'].astype(str)!='[]']
len(x)

In [ ]:
df['diam_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
diam=df[df['diam_matches'].astype(str)!='[]']
len(diam)

In [ ]:
replacement = {
    '\\?\"\s?diam': " in",
    "\\?\"\s?Diam": " in",
    '\" diam': " in",
    '\" Diam': " in",
    '\" diam': " in",
    '\\\" diam': " in",
    '\\\" diam': " in",
    'diam': " in",
    '\\': "",
    '\"': "",
}

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(3)

In [ ]:
x['matches']= x['matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(3)

# matches

In [ ]:
#import regex as re
matches=x
matches[curation_col]=matches['matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [ ]:
# # matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'01in','2.01',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'xa0','',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'inin','in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))

In [ ]:
# matches=matches[matches['buckets'].astype(str)!='Lamps']

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [136]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}-general- match upload_Shawn.csv',index=False)

In [137]:
looks_good('Bed Bath & Beyond', attribute, df, matches)

In [149]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA- {attribute}-- match upload_Shawn.csv',index=False)

In [150]:
looks_good_na('Bed Bath & Beyond', attribute, df, na_upload)